In [1]:
!pip install ultralytics --quiet --no-deps

from ultralytics import YOLO
import cv2
from pathlib import Path

MODEL_PATH = "/kaggle/input/spotify-detection-model/other/default/3/runs/segment/spotify_segmentation/weights/best.pt"

CONF_THRESHOLD = 0.55
IOU_THRESHOLD  = 0.30
MAX_DET = 20
LINE_WIDTH = 2

def process_video(model, input_video: str, output_video: str):
    cap = cv2.VideoCapture(input_video)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    out = cv2.VideoWriter(
        output_video,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h),
    )

    for r in model.predict(
        source=input_video,
        stream=True,
        conf=CONF_THRESHOLD,
        iou=IOU_THRESHOLD,
        max_det=MAX_DET,
        retina_masks=True,
        verbose=False,
    ):
        frame = r.plot(
            masks=True,
            boxes=True,
            labels=True,
            conf=True,
            line_width=LINE_WIDTH,
        )
        out.write(frame)

    out.release()
    print("Saved:", output_video)

def process_videos():
    model = YOLO(MODEL_PATH)

    videos = [
        (
            "/kaggle/input/spoti-detection/spotify-video-for-detection.mp4",
            "/kaggle/working/spotify_output_mask_1.mp4",
        ),
        (
            "/kaggle/input/spoti-detection/spotify-video-for-detection-2.mp4",
            "/kaggle/working/spotify_output_mask_2.mp4",
        ),
    ]

    for inp, outp in videos:
        print("\nProcessing:", Path(inp).name)
        process_video(model, inp, outp)

process_videos()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

Processing: spotify-video-for-detection.mp4
Saved: /kaggle/working/spotify_output_mask_1.mp4

Processing: spotify-video-for-detection-2.mp4
Saved: /kaggle/working/spotify_output_mask_2.mp4
